In [1]:
#tải các thư viện cần thiết
!pip install transformer
!pip install seqeval

ERROR: Could not find a version that satisfies the requirement transformer (from versions: none)
ERROR: No matching distribution found for transformer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=3c6b4098bd7516757e63ac8b28250bda95f0b620f3e694281b24e27f9bce038d
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:
#import các thư viện cần thiết

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import autograd

#Embedding
from transformers import AutoModel, AutoTokenizer, AutoModelForTokenClassification

#Modeling
from torch.utils.data import DataLoader
from torch.optim import SGD, Adam
from seqeval.metrics import classification_report
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [4]:
#hàm đọc dữ liệu
def read_dataset(file_path):
    tokens=[]
    ner_tags=[]
    ids=[]
    count=1
    with open(file_path) as f:
        lines=f.readlines()
        ts=[]
        nts=[]
        for line in lines:
            line = line.split()
            if len(line)==0:
                ids.append(count)
                tokens.append(ts)
                ner_tags.append(nts)
                ts=[]
                nts=[]
                count+=1
            else:
                ts.append(line[0])
                nts.append(line[-1])
    data = pd.DataFrame({'Id':ids, 'NER_tags':ner_tags, 'Tokens':tokens})
    return data

In [5]:
def get_NER_labels(data):
    NERs = list(data['NER_tags'].values)
    labels_list = []
    for value in NERs:
        labels_list = labels_list + value
    types = list(set(labels_list))
    return types

In [6]:
train_df = read_dataset('/kaggle/input/covid19vi/word/train_word.conll')
val_df = read_dataset('/kaggle/input/covid19vi/word/dev_word.conll')

In [7]:
train_df.head()

,Id,NER_tags,Tokens
0,1,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Đồng_thời, ,, bệnh_viện, tiếp_tục, thực_hiện,..."
1,2,"[O, O, O, O, O, O, O, B-SYMPTOM_AND_DISEASE, I...","["", Số, bệnh_viện, có_thể, tiếp_nhận, bệnh_nhâ..."
2,3,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[Ngoài_ra, ,, những, người, tiếp_xúc, gián_tiế..."
3,4,"[O, O, O, O, O, O, B-LOCATION, O, B-LOCATION, ...","[Bà, này, khi, trở, về, quá_cảnh, Doha, (, Qat..."
4,5,"[O, O, B-PATIENT_ID, O, O, O, O, O, O, B-PATIE...","["", Bệnh_nhân, 523, "", và, chồng, là, "", bệnh_..."


In [8]:
unique_labels = get_NER_labels(train_df)

# **TẠO LỚP DATASET BẰNG PYTORCH DATASET**

In [9]:
def align_label(text, labels, flag=False):
    label_all_tokens = flag #flag xác định cách thực hiện align_label
    
    tokenized_input = tokenizer(text, padding='max_length', max_length=256, truncation=True, is_split_into_words=True)
    word_ids = tokenized_input.input_ids
    
    start_part = True
    label_ids = []
    count = 0
    
    for i in range(len(word_ids)):
        
        if word_ids[i] == 0 or word_ids[i] == 1 or word_ids[i] == 2:
            label_ids.append(-100)
            
        elif count < len(text) and ''.join(tokenizer.decode(tokenized_input['input_ids'][i]).split()) == text[count]:
            label_ids.append(labels_to_ids[labels[count]])
            count+=1
            start_part = True
        else:
            if start_part:
                label_ids.append(labels_to_ids[labels[count]])
                count+=1
                start_part = False
            else:
                label_ids.append(labels_to_ids[labels[count]] if label_all_tokens else -100)
                
    return label_ids

class DataSet(torch.utils.data.Dataset):

    def __init__(self, df, flag_align_label=False):

        lb = df['NER_tags'].values.tolist()
        txt = df['Tokens'].values.tolist()
        self.texts = [tokenizer(i, padding='max_length', max_length = 256,
                                truncation=True, return_tensors="pt", is_split_into_words=True) for i in txt]
        self.labels = [align_label(i,j,flag_align_label) for i,j in zip(txt, lb)]
        
    def __len__(self):

        return len(self.labels)

    def get_data(self, idx):
        return self.texts[idx]

    def get_labels(self, idx):
        return torch.LongTensor(self.labels[idx])

    def __getitem__(self, idx):
        data = self.get_data(idx)
        labels = self.get_labels(idx)

        return data, labels

**Tạo 2 dictionary để xác định id nào sẽ là label nào và ngược lại.**

In [10]:
labels_to_ids = {k: (v+1) for v, k in enumerate(unique_labels)}
ids_to_labels = {(v+1): k for v, k in enumerate(unique_labels)}

In [11]:
labels_to_ids['START_TAG']=0
labels_to_ids['STOP_TAG']=len(unique_labels)+1
ids_to_labels[0]='START_TAG'
ids_to_labels[len(unique_labels)+1]='STOP_TAG'

In [12]:
labels_to_ids

{'I-SYMPTOM_AND_DISEASE': 1,
 'I-JOB': 2,
 'B-SYMPTOM_AND_DISEASE': 3,
 'I-LOCATION': 4,
 'I-TRANSPORTATION': 5,
 'B-DATE': 6,
 'B-JOB': 7,
 'I-PATIENT_ID': 8,
 'I-DATE': 9,
 'I-NAME': 10,
 'B-ORGANIZATION': 11,
 'B-TRANSPORTATION': 12,
 'B-NAME': 13,
 'I-ORGANIZATION': 14,
 'B-AGE': 15,
 'I-AGE': 16,
 'O': 17,
 'B-LOCATION': 18,
 'B-PATIENT_ID': 19,
 'B-GENDER': 20,
 'START_TAG': 0,
 'STOP_TAG': 21}

# **CÁC HÀM CHO BỘ PHÂN LOẠI CRF**

In [13]:
def log_sum_exp(vec):
    '''
    This function calculates the score explained above for the forward algorithm
    vec 2D: 1 * tagset_size
    '''
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))
    
def argmax(vec):
    '''
    This function returns the max index in a vector
    '''
    _, idx = torch.max(vec, 1)
    return to_scalar(idx)

def to_scalar(var):
    '''
    Function to convert pytorch tensor to a scalar
    '''
    return var.view(-1).data.tolist()[0]

In [14]:
def score_sentences(self, feats, tags):
    r = torch.LongTensor(range(feats.size()[0]))
    if self.use_gpu:
        r = r.cuda()
        pad_start_tags = torch.cat([torch.cuda.LongTensor([self.tag_to_ix['START_TAG']]), tags])
        pad_stop_tags = torch.cat([tags, torch.cuda.LongTensor([self.tag_to_ix['STOP_TAG']])])
    else:
        pad_start_tags = torch.cat([torch.LongTensor([self.tag_to_ix['START_TAG']]), tags])
        pad_stop_tags = torch.cat([tags, torch.LongTensor([self.tag_to_ix['STOP_TAG']])])

    score = torch.sum(self.transitions[pad_stop_tags, pad_start_tags]) + torch.sum(feats[r, tags])

    return score

In [15]:
def forward_alg(self, feats):
    '''
    This function performs the forward algorithm
    '''
    # calculate in log domain
    # feats is len(sentence) * tagset_size
    # initialize alpha with a Tensor with values all equal to -10000.
    
    # Do the forward algorithm to compute the partition function
    init_alphas = torch.Tensor(1, self.tagset_size).fill_(-10000.)
    
    # START_TAG has all of the score.
    init_alphas[0][self.tag_to_ix['START_TAG']] = 0.
    
    # Wrap in a variable so that we will get automatic backprop
    forward_var = autograd.Variable(init_alphas)
    if self.use_gpu:
        forward_var = forward_var.cuda()
        
    # Iterate through the sentence
    for feat in feats:
        # broadcast the emission score: it is the same regardless of
        # the previous tag
        emit_score = feat.view(-1, 1)
        
        # the ith entry of trans_score is the score of transitioning to
        # next_tag from i
        tag_var = forward_var + self.transitions + emit_score
        
        # The ith entry of next_tag_var is the value for the
        # edge (i -> next_tag) before we do log-sum-exp
        max_tag_var, _ = torch.max(tag_var, dim=1)
        
        # The forward variable for this tag is log-sum-exp of all the
        # scores.
        tag_var = tag_var - max_tag_var.view(-1, 1)
        
        # Compute log sum exp in a numerically stable way for the forward algorithm
        forward_var = max_tag_var + torch.log(torch.sum(torch.exp(tag_var), dim=1)).view(1, -1) # ).view(1, -1)
    terminal_var = (forward_var + self.transitions[self.tag_to_ix['STOP_TAG']]).view(1, -1)
    alpha = log_sum_exp(terminal_var)
    # Z(x)
    return alpha

In [16]:
def get_neg_log_likelihood(self, feats, tags):
    # sentence, tags is a list of ints
    # features is a 2D tensor, len(sentence) * self.tagset_size

    forward_score = self._forward_alg(feats)
    gold_score = self._score_sentence(feats, tags)
    return forward_score - gold_score

# **BUILD MODEL PHOBERT FOR TOKEN CLASSIFICATION**

In [17]:
class PhoBertModel(torch.nn.Module):

    def __init__(self):

        super(PhoBertModel, self).__init__()
        
        self.tag_to_ix = labels_to_ids
        self.use_gpu = True
        self.tagset_size = len(labels_to_ids)
        self.transitions = nn.Parameter(torch.zeros(self.tagset_size, self.tagset_size))
        self.transitions.data[labels_to_ids['START_TAG'], :] = -10000
        self.transitions.data[:, labels_to_ids['STOP_TAG']] = -10000
        self.phobert = AutoModelForTokenClassification.from_pretrained("vinai/phobert-base", num_labels=(len(unique_labels)+2))

    def forward(self, input_id, mask, label):
        
        output = self.phobert(input_ids=input_id, attention_mask=mask, labels=label, return_dict=False)
        
        return output
    neg_log_likelihood = get_neg_log_likelihood
    _forward_alg = forward_alg
    _score_sentence = score_sentences

# **TRAIN MODEL**

In [18]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [19]:
def train_loop(model, train_df, val_df, flag_align_label):

    train_dataset = DataSet(train_df, flag_align_label)
    val_dataset = DataSet(val_df, flag_align_label)

    train_dataloader = DataLoader(train_dataset, num_workers=4, batch_size=BATCH_SIZE, shuffle=True)
    val_dataloader = DataLoader(val_dataset, num_workers=4, batch_size=BATCH_SIZE)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    optimizer = Adam(model.parameters(), lr=LEARNING_RATE)

    if use_cuda:
        model = model.cuda()
        
    min_val_loss = 1000
    count = 0
    
    for epoch_num in range(EPOCHS):

        total_loss_train = 0

        model.train()

        for train_data, train_label in tqdm(train_dataloader):

            train_label = train_label.to(device)
            mask = train_data['attention_mask'].squeeze(1).to(device)
            input_id = train_data['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            _, logits = model(input_id, mask, train_label)


            logits_clean = logits[0][train_label[0] != -100]
            label_clean = train_label[0][train_label[0] != -100]

            loss=model.neg_log_likelihood(logits_clean, label_clean)
            total_loss_train += loss.data
            loss.backward()
            optimizer.step()

        model.eval()

        total_loss_val = 0

        for val_data, val_label in val_dataloader:

            val_label = val_label.to(device)
            mask = val_data['attention_mask'].squeeze(1).to(device)
            input_id = val_data['input_ids'].squeeze(1).to(device)

            _, logits = model(input_id, mask, val_label)

            logits_clean = logits[0][val_label[0] != -100]
            label_clean = val_label[0][val_label[0] != -100]
            loss=model.neg_log_likelihood(logits_clean, label_clean)
            total_loss_val += loss.data


        print(
            f'Epochs: {epoch_num + 1} | Loss: {total_loss_train / len(train_df): .3f} | Val_Loss: {total_loss_val / len(val_df): .3f}')
        if (total_loss_val / len(val_df)) < min_val_loss:
            min_val_loss = total_loss_val / len(val_df)
            torch.save(model.state_dict(), 'phobert_base_crf_ner')
            count = epoch_num
        if epoch_num - count >= 5:
            return
        
LEARNING_RATE = 5e-5
EPOCHS = 30
BATCH_SIZE = 1

model = PhoBertModel()
train_loop(model, train_df, val_df, False)

pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 5027/5027 [05:44<00:00, 14.59it/s]


Epochs: 1 | Loss:  3.810 | Val_Loss:  2.194


100%|██████████| 5027/5027 [05:44<00:00, 14.59it/s]


Epochs: 2 | Loss:  1.455 | Val_Loss:  2.393


...

Epochs: 8 | Loss:  1.048 | Val_Loss:  2.414


100%|██████████| 5027/5027 [05:41<00:00, 14.71it/s]


Epochs: 9 | Loss:  0.682 | Val_Loss:  3.192


100%|██████████| 5027/5027 [05:41<00:00, 14.71it/s]


Epochs: 10 | Loss:  0.661 | Val_Loss:  2.736


# **EVALUATE MODEL**

In [20]:
test_df = read_dataset('/kaggle/input/covid19vi/word/test_word.conll')

In [21]:
def evaluate(model, test_df, flag_align_label, ids_to_labels):

    test_dataset = DataSet(test_df, flag_align_label)

    test_dataloader = DataLoader(test_dataset, num_workers=4, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    predictions = []
    labels = []
    
    for test_data, test_label in test_dataloader:

            test_label = test_label.to(device)
            mask = test_data['attention_mask'].squeeze(1).to(device)

            input_id = test_data['input_ids'].squeeze(1).to(device)

            loss, logits = model(input_id, mask, test_label)

            for i in range(logits.shape[0]):
                cleaned_logits = logits[i][test_label[i] != -100].argmax(dim=1)
                predictions.append([ids_to_labels[val.item()] for val in cleaned_logits])
                cleaned_labels = test_label[i][test_label[i] != -100]
                labels.append([ids_to_labels[val.item()] for val in cleaned_labels])
    print(classification_report(y_pred=predictions, y_true=labels, digits=3))

evaluate(model, test_df, False, ids_to_labels)

                     precision    recall  f1-score   support

                AGE      0.965     0.981     0.973       568
               DATE      0.964     0.989     0.977      1641
             GENDER      0.943     0.978     0.960       458
                JOB      0.725     0.657     0.689       169
           LOCATION      0.924     0.951     0.938      4346
               NAME      0.940     0.895     0.917       314
       ORGANIZATION      0.806     0.854     0.829       768
         PATIENT_ID      0.970     0.984     0.977      1982
SYMPTOM_AND_DISEASE      0.803     0.862     0.831      1135
     TRANSPORTATION      0.963     0.974     0.968       189

          micro avg      0.918     0.944     0.931     11570
          macro avg      0.900     0.912     0.906     11570
       weighted avg      0.919     0.944     0.931     11570



# **INFERENCE**

In [22]:
def align_word_ids(text, flag):
    label_all_tokens = flag
    
    text = text.split()
  
    tokenized_inputs = tokenizer(text, padding='max_length', max_length=256, truncation=True, is_split_into_words=True)

    word_ids = tokenized_inputs.input_ids

    start_part = True
    label_ids = []
    count = 0
    
    for i in range(len(word_ids)):
        
        if word_ids[i] == 0 or word_ids[i] == 1 or word_ids[i] == 2:
            label_ids.append(-100)
            
        elif count < len(text) and ''.join(tokenizer.decode(tokenized_inputs['input_ids'][i]).split()) == text[count]:
            label_ids.append(1)
            count+=1
            start_part = True
        else:
            if start_part:
                label_ids.append(1)
                count+=1
                start_part = False
            else:
                label_ids.append(1 if label_all_tokens else -100)           
    return label_ids

In [23]:
def ner(model, sentence, flag_align_label):

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:
        model = model.cuda()
    text = tokenizer(sentence, padding='max_length', max_length = 256, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence, flag_align_label)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)

In [25]:
ner(model,
    'Bệnh nhân nhập viện tối qua ở Bệnh Viện 115 là bệnh nhân thứ 82 , di chuyển qua nhiều thành phố bằng xe biển hiệu E-402',
    flag_align_label=False)

Bệnh nhân nhập viện tối qua ở Bệnh Viện 115 là bệnh nhân thứ 82 , di chuyển qua nhiều thành phố bằng xe biển hiệu E-402
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'I-LOCATION', 'I-LOCATION', 'O', 'O', 'O', 'O', 'B-PATIENT_ID', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-TRANSPORTATION']
